In [1]:
import getpass
import os

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = getpass.getpass("Enter your Lang API key: ")

In [ ]:
import getpass
import os

os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter your OpenAI API key: ")

from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-3.5-turbo-0125")


In [1]:
from langchain_core.messages import HumanMessage, SystemMessage

messages = [
  SystemMessage(content="Translate the following from English into Italian"),
  HumanMessage(content="hi!")
]

# just call the model on the list of messages
# which contains one message from the system (with the directives to follow)
# and one message from the user
model.invoke(messages)

# the model is a runnable (it has the invoke method)

NameError: name 'model' is not defined

In [ ]:
from langchain_core.output_parsers import StrOutputParser

# the output above containts lot of metadata, we are just interested in the text
str_parser = StrOutputParser()

In [ ]:
# my first chain! I just chain the model with the output parser
chain = model | str_parser

chain.invoke(messages)

# the chain is a runnable too

'Ciao!'

Right now we are passing a list of messages directly into the language model. Where does this list of messages come from? Usually, it is constructed from a combination of user input and application logic. This application logic usually takes the raw user input and transforms it into a list of messages ready to pass to the language model. Common transformations include adding a system message or formatting a template with the user input.

PromptTemplates are a concept in LangChain designed to assist with this transformation. They take in raw user input and return data (a prompt) that is ready to pass into a language model.

In [8]:
from langchain_core.prompts import ChatPromptTemplate

system_template = "Translate the following into {language}:"

prompt_template = ChatPromptTemplate.from_messages(
  [
    ("system", system_template), # becomes a SystemMessage
    ("user", "{text}"), # becomes a HumanMessage
  ]
)

# there are two variables here:
# - language (which is the target language)
# - text (which is the text to translate)


# guess who is a runnable too?
ptr = prompt_template.invoke({
  "language": "italian",
  "text": "hi there, how are you?"
})

ptr.to_messages()

[SystemMessage(content='Translate the following into italian:', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='hi there, how are you?', additional_kwargs={}, response_metadata={})]

In [10]:
# the final chain
final_chain = prompt_template | model | str_parser

final_chain.invoke({
  "language": "italian",
  "text": "hi there, how are you?"
})

'ciao, come stai?'

This is a simple example of using LangChain Expression Language (LCEL) to chain together LangChain modules. There are several benefits to this approach, including optimized streaming and tracing support.

If we take a look at the LangSmith trace, we can see all three components show up in the LangSmith trace.
